### Часть 1.1
### Реализация парсера

In [1]:
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from dataclasses import dataclass
import seaborn as sns
import pandas as pd
import random
import requests
import nltk
nltk.download("stopwords")

sns.set(style="darkgrid")
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kukof\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
DEPTH = 20
SLEEP = random.randint(1,5)
BASE_URL = 'https://news.mail.ru/'
TOPICS = ['economics','politics' , 'society', 'incident']

In [3]:
@dataclass
class Article:
    id: str = None
    url: str = None
    source: str = None
    title: str = None
    content: str = None
    datetime: str = None

In [4]:
# set webdriver params
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('no-sandbox')
chrome_options.add_argument('disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

# driver = webdriver.Chrome('./chromedriver')

In [5]:
def get_pages():

    """Load and scroll pages"""

    items, topics_order = [], []

    for topic in tqdm(TOPICS):
        try:
            old_size = len(items)
            URL = BASE_URL + topic
            driver.get(URL)
            time.sleep(SLEEP)

     # We can adjust this number to get more posts
            last_height = driver.execute_script("return document.body.scrollHeight")

            NUM_SCROLLS = 5000

            for i in range(NUM_SCROLLS):

                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(SLEEP)
                driver.execute_script("document.getElementsByClassName('button js-pgng_more_link')[0].click()")
                time.sleep(SLEEP)

                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            items += soup.find_all('a', {'class' : 'newsitem__title link-holder'})
            new_size = len(items)
            if new_size > old_size:
                topics_order.extend([topic] * (new_size - old_size))

        except:
            pass

    return items, topics_order



In [6]:
def parse_page(page):
    """Extract from page desired fields"""

    # Create article data class object
    article = Article()

    # article id
    article.url = page['href']

    # article url
    article.id = article.url[30: article.url.find('/')]

    # load page
    driver.get(article.url)
    time.sleep(random.randint(1,5))
    html = driver.page_source

    #soup = BeautifulSoup(html, "html.parser")
    response = requests.get(article.url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # article source
    article.source = article.url[12: article.url.find('.')]

    # process article title
    article.title = soup.find('h1', {'class': 'hdr__inner'}).text


    # article content
    article.content = soup.find('div', {'class': 'article__text js-module js-view js-media-stat-article js-smoky-links'}).text

    # article datetime
    str_ = soup.find('span', {'class': 'note__text breadcrumbs__text js-ago'})['datetime']
    article.datetime = str_[:str_.find('T')]

    return article

In [7]:

# get pages and topics
pages, topics_order = get_pages()
# with open('pages.txt', 'w') as f:
#     for page in pages:
#         f.write("%s\n" % page)

100%|██████████| 4/4 [56:31<00:00, 847.75s/it]


In [8]:
pages

[<a class="newsitem__title link-holder" href="https://news.mail.ru/economics/58474302/"><span class="newsitem__title-inner">Названы самые популярные профессии в Омске в октябре 2023 года</span></a>,
 <a class="newsitem__title link-holder" href="https://finance.mail.ru/2023-11-03/novostroyki-uhodyat-v-otryv-chem-opasen-disbalans-cen-na-pervichnom-i-vtorichnom-rynkah-58479084/"><span class="newsitem__title-inner">Новостройки уходят в отрыв. Чем опасен дисбаланс цен на первичном и вторичном рынках недвижимости</span></a>,
 <a class="newsitem__title link-holder" href="https://finance.mail.ru/2023-11-03/rspp-predlozhil-povysit-potolok-dohoda-dlya-samozanyatyh-58479590/"><span class="newsitem__title-inner">РСПП предложил повысить потолок дохода для самозанятых</span></a>,
 <a class="newsitem__title link-holder" href="https://finance.mail.ru/2023-11-03/dollary-ne-poddayutsya-perevodu-kak-novye-sankcii-ssha-otrazyatsya-na-deyatelnosti-58478073/"><span class="newsitem__title-inner">Доллары не п

In [9]:
# parse each page and get desired attributes
   # few pages are differ significantly from others
# using try/except we can ignore them
data, topics_order_fixed = [], []
for num, page in enumerate(tqdm(pages)):
    try:
        res = parse_page(page)
        data.append(res)
        topics_order_fixed.append(topics_order[num])
    except:
        pass

driver.close()

100%|██████████| 14471/14471 [19:34:54<00:00,  4.87s/it]  


In [39]:
df_mail = pd.DataFrame(data=data)
df_mail['topic'] = topics_order_fixed
df_mail

,id,url,source,title,content,datetime,topic
0,,https://news.mail.ru/economics/58474302/,,Названы самые популярные профессии в Омске в о...,Источник: FreepikАналитики отмечают: чаще всег...,2023-11-03,economics
1,,https://news.mail.ru/economics/58468958/,,Роспатент опроверг «конфискацию» товарных знак...,"Источник: РИА ""Новости""Ранее финская газета ES...",2023-11-02,economics
2,,https://news.mail.ru/economics/58457933/,,Правительство побудит регионы пересмотреть сис...,Источник: Фотоархив ИД «Коммерсантъ» В России ...,2023-11-02,economics
3,,https://news.mail.ru/economics/58456550/,,Путин рассказал о росте ВВП России за девять м...,"Источник: РИА ""Новости""«Оперативная статистика...",2023-11-01,economics
4,,https://news.mail.ru/economics/58455328/,,Автоконцерны попросили власти помочь с проблем...,Источник: ReutersПо информации одного из собес...,2023-11-01,economics
...,...,...,...,...,...,...,...
10759,,https://news.mail.ru/incident/56069708/,,Истребитель ВКС России уничтожил украинский са...,Во время загрузки произошла ошибка.На кадрах в...,2023-05-03,incident
10760,,https://news.mail.ru/incident/56070824/,,ФСБ сообщила о предотвращении покушения на Акс...,ФСБ пресекла подготовку покушений на руководит...,2023-05-03,incident
10761,,https://news.mail.ru/incident/56070411/,,Названа причина возгорания резервуара с нефтеп...,Во время загрузки произошла ошибка.Об этом ТАС...,2023-05-03,incident
10762,,https://news.mail.ru/incident/56070013/,,Губернатор Кубани опроверг сообщения об эвакуа...,Во время загрузки произошла ошибка.— Сейчас в ...,2023-05-03,incident


In [41]:
df_mail.duplicated().sum(), len(df_mail.drop_duplicates())

(0, 10764)

In [44]:
df_mail.source = 'news.mail'

In [47]:
df_mail.head()

,id,url,source,title,content,datetime,topic
0,,https://news.mail.ru/economics/58474302/,news.mail,Названы самые популярные профессии в Омске в о...,Источник: FreepikАналитики отмечают: чаще всег...,2023-11-03,economics
1,,https://news.mail.ru/economics/58468958/,news.mail,Роспатент опроверг «конфискацию» товарных знак...,"Источник: РИА ""Новости""Ранее финская газета ES...",2023-11-02,economics
2,,https://news.mail.ru/economics/58457933/,news.mail,Правительство побудит регионы пересмотреть сис...,Источник: Фотоархив ИД «Коммерсантъ» В России ...,2023-11-02,economics
3,,https://news.mail.ru/economics/58456550/,news.mail,Путин рассказал о росте ВВП России за девять м...,"Источник: РИА ""Новости""«Оперативная статистика...",2023-11-01,economics
4,,https://news.mail.ru/economics/58455328/,news.mail,Автоконцерны попросили власти помочь с проблем...,Источник: ReutersПо информации одного из собес...,2023-11-01,economics


In [50]:
df_mail.topic.value_counts()

topic
politics     4131
incident     3472
society      2748
economics     413
Name: count, dtype: int64

In [ ]:
df_mail.to_pickle('df_mail.p', compression='gzip')